## DB PROJECT

This project is non-profit, personal project for education purposes only.

1) scrape comics db data - save them into file (huge amount of data scraped from nonprofit db - save it to the csv and avoid another round of scraping)
2) create sqlite3 db (separate comics.sql script)
3) import data from scraped .csv files
4) connect a python command line tool to the db and allow users to search the db

Thanks to the people from https://www.comicsdb.cz/ project.

### Importing all the neccessary libraries for scraping

In [4]:
import csv
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

DRIVER_PATH = "C:\\Users\\petr.musil\\Desktop\\python\\edgedriver\\msedgedriver.exe"
service = Service(executable_path = DRIVER_PATH)

### Data collection - download all of the publishers links from the comicsdb.cz and export them to a .csv file

In [5]:
try:
    driver = webdriver.Edge(service=service)
    publisher = "https://www.comicsdb.cz/prehled-nakladatelstvi/1/"
    list_of_links = []
    for num in list(range(1,8)):
        driver.get(f'{publisher}{num}')    
        time.sleep(1)
        # wait for the element
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".card.text-center.p-2")))
        links = driver.find_element(by = By.CSS_SELECTOR, value = ".table-border-dashed").find_elements(by = By.TAG_NAME, value = "a")
        for link in links:
            list_of_links.append([link.get_attribute("href")])
finally:
    driver.quit()

filename = "./data/links_publishers.csv"
with open(filename, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(list_of_links)

### Visit each webpage of a publisher from the source .csv file and get name, number of titles and link. Save the result to a .csv file

In [29]:
try:
    list_of_publisher_data = []
    with open("./data/links_publishers.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            driver.get(row[0])
            time.sleep(1)
            publisher = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".font-weight-semibold")))
            number_of_titles = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".font-weight-bold.text-cdbred.ml-1")))
            list_of_publisher_data.append([publisher.text.strip(), number_of_titles.text.strip(), row[0].strip()])
finally:
    driver.quit()

filename = "./data/output_publishers_data.csv"
with open(filename, "w", newline="", encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=";")
    csvwriter.writerow(["name", "number_of_titles", "link"])
    csvwriter.writerows(list_of_publisher_data)

### Visit all titles overviews and save links to all the titles to a .csv file

In [39]:
try:
    overview = "https://www.comicsdb.cz/prehled-comicsu/6/"
    list_of_links = []
    for num in list(range(1,106)):
        driver.get(f'{overview}{num}')    
        time.sleep(2)
        # wait for the element
        table_ = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".table.table-hover.table-xs")))
        links = driver.find_elements(by = By.CSS_SELECTOR, value = ".table-border-dashed a")
        #links = links.find_elements(by = By.TAG_NAME, value = "a")
        for link in links:
            list_of_links.append([link.get_attribute("href")])
finally:
    driver.quit()

filename = "./data/links_titles.csv"
with open(filename, "w", newline="", encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(list_of_links)

### Visit all the titles details and get all the data

In [7]:
try:
    list_of_titles_data = []
    with open("./data/links_titles.csv", "r") as file:
        reader = csv.reader(file)
        for row in reader:
            driver.get(row[0])
            time.sleep(2)
            title = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".font-weight-semibold")))
            soup = BeautifulSoup(driver.page_source, "html.parser")
            title = soup.find("span", "font-weight-semibold").text
            basic_info = soup.find("small")
            try:
                link_publisher = basic_info.find("a").get("href")
            except:
                link_publisher = "NA"
            try:
                year_pattern = re.compile(r'\d{4}')
                year = year_pattern.search(basic_info.text)
                year = year.group()
            except:            
                year = "NA"
            pages = "NA"
            price = "NA"
            dt_tags = soup.find_all("dt")
            for tag in dt_tags:
                if "Stran" in tag.text:
                    pages = tag.next_sibling.text
                elif "Cena" in tag.text:
                    price = tag.next_sibling.text
            list_of_titles_data.append([title.strip(), year.strip(), row[0].strip(), link_publisher.strip(), pages.strip(), price.strip()])
finally:
    driver.quit()

filename = "./data/output_titles_data.csv"
with open(filename, "w", newline="", encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=";")
    csvwriter.writerow(["title", "year", "link_title","link_publisher", "pages", "price"])
    csvwriter.writerows(list_of_titles_data)
    

### Clean the data from csv files, restructure them and import to sqlite3 db

In [45]:
import pandas as pd
import sqlite3
from urllib.parse import urlparse

titles = pd.read_csv("./data/output_titles.csv", delimiter=";")
publishers = pd.read_csv("./data/output_publishers_data.csv", delimiter=";")

titles_origin = titles.copy()
titles = titles[titles["link_publisher"] == "/nakladatelstvi/1/crew"]
titles["year"] = titles["year"].astype(int)
mean_pages = titles["pages"].mean()
titles["pages"] = titles["pages"].fillna(mean_pages).astype(int)
titles["price"] = titles["price"].str.replace(" Kč", "").astype(int)
titles_origin["year"] = titles_origin["year"].fillna(0)
titles_origin["year"] = titles_origin["year"].astype(int)
mean_pages = titles_origin["pages"].mean()
titles_origin["pages"] = titles_origin["pages"].fillna(mean_pages).astype(int)
titles_origin["price"] = titles_origin["price"].fillna(0)
titles_origin["price"] = titles_origin["price"].str.replace(" Kč", "")
titles_origin["price"] = titles_origin["price"].str.replace(" h", "")
titles_origin["price"] = titles_origin["price"].str.replace(" K", "")
titles_origin["price"] = titles_origin["price"].str.replace(" k", "")
titles_origin["price"] = titles_origin["price"].str.replace(" Lei", "")
titles_origin["price"] = titles_origin["price"].str.strip().astype(float)

# Save some space - get the same part of url as in the titles csv file.
publishers["link"] = publishers["link"].apply(lambda x: urlparse(x).path)
publishers.rename(columns = {"index": "publisher_id"}, inplace = True)

# import publishers to the db - let them get their pubsliher_id in the db and export, then joinig with titles from csv and fetch the pubslisher_id to the titles
try:
    conn = sqlite3.connect("./db/comics.db")
    cur = conn.cursor()
    # import publishers df to the sqlite3 db, change the name of index to publisher_id
    publishers.to_sql('publishers', conn, if_exists='replace', index=True, index_label='publisher_id')
    result = cur.execute("SELECT * FROM publishers;")
    rows = result.fetchall()
    columns = [column[0] for column in cur.description]
    publishers_final = pd.DataFrame(rows, columns=columns)
    titles_origin_final = titles_origin.merge(publishers_final, how="left", left_on="link_publisher", right_on="link")
    titles_origin_final.drop(["name", "number_of_titles", "link"],  axis = 1, inplace=True)
    # renaming column
    titles_origin_final.rename(columns = {"index": "publisher_id"}, inplace = True)
    # if there is no publisher assign some special id
    titles_origin_final["publisher_id"] = titles_origin_final["publisher_id"].fillna(999999)
    titles_origin_final["publisher_id"] = titles_origin_final["publisher_id"].astype(int)
    titles_origin_final.to_sql('titles', conn, if_exists='replace', index=True, index_label='title_id')
finally:
    conn.close()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10453 entries, 0 to 10452
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           10453 non-null  object 
 1   year            10453 non-null  int32  
 2   link_title      10453 non-null  object 
 3   link_publisher  10391 non-null  object 
 4   pages           10453 non-null  int32  
 5   price           9807 non-null   float64
 6   publisher_id    10453 non-null  int32  
dtypes: float64(1), int32(3), object(3)
memory usage: 530.8+ KB


### Explore the gathered data

In [47]:
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

def fetch_df_from_db(cursor, sql):
    """Gets cursor and SQL code and runs the SQL code on the cursor. Returns ready dataframe from the db data."""
    result = cursor.execute(sql)
    rows = result.fetchall()
    columns = [column[0] for column in cursor.description]
    return pd.DataFrame(rows, columns=columns)

try:
    conn = sqlite3.connect("./db/comics.db")
    cur = conn.cursor()
    titles_db = fetch_df_from_db(cur, "SELECT * FROM titles;")
    publishers_db = fetch_df_from_db(cur, "SELECT * FROM publishers;")
    print(titles_db.info(), publishers_db.info())
finally:
    conn.close()

"""
publishers = pd.read_csv("./data/output_publishers_data.csv", delimiter=";")
publishers = publishers[publishers["number_of_titles"] > 100]
publishers.plot(kind = "bar", x = "name", y = "number_of_titles")
plt.show() """

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10453 entries, 0 to 10452
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title_id        10453 non-null  int64  
 1   title           10453 non-null  object 
 2   year            10453 non-null  int64  
 3   link_title      10453 non-null  object 
 4   link_publisher  10391 non-null  object 
 5   pages           10453 non-null  int64  
 6   price           9807 non-null   float64
 7   publisher_id    10453 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 653.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   publisher_id      618 non-null    int64 
 1   name              618 non-null    object
 2   number_of_titles  618 non-null    int64 
 3   link              618 non-null    objec

'\npublishers = pd.read_csv("./data/output_publishers_data.csv", delimiter=";")\npublishers = publishers[publishers["number_of_titles"] > 100]\npublishers.plot(kind = "bar", x = "name", y = "number_of_titles")\nplt.show() '